<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Face-Extraction" data-toc-modified-id="Face-Extraction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Face Extraction</a></span><ul class="toc-item"><li><span><a href="#MTCNN" data-toc-modified-id="MTCNN-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>MTCNN</a></span></li></ul></li><li><span><a href="#Face-Alignment" data-toc-modified-id="Face-Alignment-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Face Alignment</a></span><ul class="toc-item"><li><span><a href="#Landmark-Detection" data-toc-modified-id="Landmark-Detection-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Landmark Detection</a></span></li><li><span><a href="#Find-Convex-Hull" data-toc-modified-id="Find-Convex-Hull-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Find Convex Hull</a></span></li><li><span><a href="#Delaunay-Triangulation" data-toc-modified-id="Delaunay-Triangulation-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Delaunay Triangulation</a></span><ul class="toc-item"><li><span><a href="#Plot-Triangulation" data-toc-modified-id="Plot-Triangulation-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Plot Triangulation</a></span></li></ul></li><li><span><a href="#Affine-warp-triangles" data-toc-modified-id="Affine-warp-triangles-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Affine warp triangles</a></span></li><li><span><a href="#Align-Matrix" data-toc-modified-id="Align-Matrix-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Align Matrix</a></span></li></ul></li><li><span><a href="#Face-Blending" data-toc-modified-id="Face-Blending-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Face Blending</a></span><ul class="toc-item"><li><span><a href="#Mask" data-toc-modified-id="Mask-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Mask</a></span></li></ul></li><li><span><a href="#Face-Generation" data-toc-modified-id="Face-Generation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Face Generation</a></span><ul class="toc-item"><li><span><a href="#Autoencoder-Generation" data-toc-modified-id="Autoencoder-Generation-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Autoencoder Generation</a></span></li></ul></li></ul></div>

# Intro
Notebook exploring face-swap in Python.

Resources:
* [Face Swap using OpenCV](https://www.learnopencv.com/face-swap-using-opencv-c-python/)
* [How to install dlib](https://www.pyimagesearch.com/2017/03/27/how-to-install-dlib/)
* [Detect eyes, nose, lips, and jaw with dlib, OpenCV, and Python](https://www.pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
import sys
import yaml

import dlib
import cv2

from tqdm import tqdm

# Plotting
%matplotlib notebook
%matplotlib inline

sns.set_context("paper")
sns.set_style("dark")

sys.path.append('../face_swap')

from utils import image_processing
import FaceGenerator
from face_swap import faceswap_utils as utils
import FaceGenerator, FaceDetector
import autoencoder
import gan
from face_swap import CONFIG_PATH
from Face import Face
import tensorflow as tf
from deep_swap import swap_faces

%load_ext autoreload
%autoreload 2

In [ ]:
data_folder = Path.home() / "Documents/datasets/"
models_folder = Path.home() / "Documents/models/"

# Load Data

In [ ]:
# Load two random celeba faces
from_face_img = cv2.cvtColor(cv2.imread(str(data_folder / "img_align_celeba" / 
                            "000{}{}{}.jpg".format(*np.random.randint(0, 9, 3)))),
                             cv2.COLOR_BGR2RGB)
to_face_img = cv2.cvtColor(cv2.imread(str(data_folder / "img_align_celeba" / 
                          "000{}{}{}.jpg".format(*np.random.randint(0, 9, 3)))),
                       cv2.COLOR_BGR2RGB)

In [ ]:
# Load faces with open-cv
from_face_img = cv2.cvtColor(cv2.imread(str(data_folder / "face_swap" / "sources" /
                            "steve.jpg")),
                             cv2.COLOR_BGR2RGB)
to_face_img = cv2.cvtColor(cv2.imread(str(data_folder / "face_swap" / "sources" /
                            "carell.png")),
                             cv2.COLOR_BGR2RGB)

In [ ]:
# Load trump/cage faces
from_face_img = cv2.cvtColor(cv2.imread(str(data_folder / "face_swap" / "legolas_01" / "frames" /
                            "frame_0001.png")),
                             cv2.COLOR_BGR2RGB)
to_face_img = cv2.cvtColor(cv2.imread(str(data_folder / "face_swap" / "legolas_04" / "frames" /
                            "frame_0007.png")),
                             cv2.COLOR_BGR2RGB)

In [ ]:
# Load trump/cage faces
from_face_img = cv2.cvtColor(cv2.imread(str(data_folder / "face_swap" / "test"/ "multi" /
                            "multi_01.jpg")),
                             cv2.COLOR_BGR2RGB)
to_face_img = cv2.cvtColor(cv2.imread(str(data_folder / "face_swap" / "test" / "multi" /
                            "multi_02.jpg")),
                             cv2.COLOR_BGR2RGB)

In [ ]:
from_face_img.shape

In [ ]:
plt.imshow(from_face_img)
plt.show()
plt.imshow(to_face_img)
plt.show()

In [ ]:
landmark_detector = utils.LandmarkDetector(str(models_folder / 'face_recognition' / 'shape_predictor_68_face_landmarks.dat'))

In [ ]:
mmod = dlib.cnn_face_detection_model_v1(str(models_folder / 'face_recognition' / 'mmod_human_face_detector.dat'))
resnet = dlib.face_recognition_model_v1(str(models_folder / 'face_recognition' / 'dlib_face_recognition_resnet_model_v1.dat'))

In [ ]:
mmod(to_face_img)[0].rect

# Face Extraction

In [ ]:
with open(CONFIG_PATH, 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

tf.reset_default_graph()
cfg['extract']['align'] = True
cfg['extract']['masked'] = False
face_detector = FaceDetector.FaceDetector(cfg)
#face_generator = FaceGenerator.FaceGenerator(lambda x : x)

In [ ]:
faces = face_detector.detect_faces(from_face_img)
aligned_face_img = face_detector.extract_face(faces[0])

In [ ]:
len(faces)

In [ ]:
plt.imshow(faces[4].face_img)
plt.show()

In [ ]:
plt.imshow(faces[1].face_img)
plt.show()

# Face Alignment

## Landmark Detection

In [ ]:
# get face boundary points and containing rectangles
# for both faces
face_boundary_from, rect_from = landmark_detector.get_contour(from_face)
face_boundary_to, rect_to = landmark_detector.get_contour(to_face)

## Find Convex Hull
Get convex hull indexes only of target face, and obtain hull points for both faces using such indexes.

In [ ]:
#hull_idx_from = cv2.convexHull(face_boundary_from, returnPoints = False)
hull_idx_to = cv2.convexHull(face_boundary_to, returnPoints = False)

In [ ]:
#??Do not use directly this cause you might lose correspondence
#between number of points
#hull_from = cv2.convexHull(face_boundary_from, returnPoints = True)
#hull_to = cv2.convexHull(face_boundary_to, returnPoints = True)

In [ ]:
hull_from = np.array([face_boundary_from[hull_idx] for hull_idx in hull_idx_to])
hull_to = np.array([face_boundary_to[hull_idx] for hull_idx in hull_idx_to])

## Delaunay Triangulation

In [ ]:
#triangles_from_idxs = utils.get_triangles_indexes(from_face, rect_from, hull_idx_to, face_boundary_from)
triangles_to_idxs = utils.get_triangles_indexes(to_face, rect_to, hull_idx_to, face_boundary_to)

In [ ]:
triangles_from = utils.clean_triangles((0, 0, from_face.shape[1], from_face.shape[0]), 
                                 utils.delaunay_triangulation(from_face, rect_from, hull_idx_from, face_boundary_from))
triangles_to = utils.clean_triangles((0, 0, to_face.shape[1], to_face.shape[0]), 
                               utils.delaunay_triangulation(to_face, rect_to, hull_idx_to, face_boundary_to))

In [ ]:
print(len(triangles_from))
print(len(triangles_to))

### Plot Triangulation

In [ ]:
import matplotlib.patches as patches

In [ ]:
# Create figure and axes
fig,ax = plt.subplots(1)

# Display the image
ax.imshow(from_face)

for t in triangles_from:
    triangle = patches.Polygon(t, linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(triangle)
# Create a Rectangle patch
#rect = patches.Rectangle((50,100),40,30,linewidth=1,edgecolor='r',facecolor='none')

# Add the patch to the Axes
#ax.add_patch(rect)
plt.show()

In [ ]:
# Create figure and axes
fig,ax = plt.subplots(1)
# Display the image
ax.imshow(to_face)

for t in triangles_to:
    triangle = patches.Polygon(t, linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(triangle)
# Create a Rectangle patch
#rect = patches.Rectangle((50,100),40,30,linewidth=1,edgecolor='r',facecolor='none')

# Add the patch to the Axes
#ax.add_patch(rect)
plt.show()

In [ ]:
np.array(triangles_to[0])

## Affine warp triangles 

In [ ]:
img_res = utils.image_affine_warp(hull_from,
                      hull_to,
                      triangles_to_idxs, 
                      from_face, 
                      to_face.copy())

In [ ]:
plt.imshow(img_res)
plt.show()

## Align Matrix

In [ ]:
matrix = utils.align_face(faces[4])

In [ ]:
aligned_face = utils._align_face(faces[4])
plt.imshow(aligned_face)
plt.show()

In [ ]:
eyes_center, angle, scale = utils.get_rotation_info(faces[4])
m = cv2.getRotationMatrix2D(faces[4].get_face_center(absolute=False), -angle, 2-scale)

In [ ]:
faces[4].get_face_center(absolute=False)

In [ ]:
plt.imshow(faces[4].face_img)
plt.show()

In [ ]:
plt.imshow(cv2.warpAffine(aligned_face, m, (63, 74),
                            flags=cv2.INTER_CUBIC))
plt.show()

In [ ]:
plt.imshow(utils.align_face(faces[4]))
plt.show()

# Face Blending

In [ ]:
with open(CONFIG_PATH, 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

#tf.reset_default_graph()
cfg['extract']['align'] = True
face_detector = FaceDetector.FaceDetector(cfg)
model_cfg = cfg['base_gan']['v0']
aut_a, aut_b, _, _ = gan.get_gan(model_cfg)
target_aut = aut_a if cfg.get('use_aut_a') else aut_b
face_generator = FaceGenerator.FaceGenerator(
        lambda seed_face, size: FaceGenerator.aue_generate_face(target_aut,
                                                                seed_face, size,
                                                                tanh_fix=True,
                                                                masked=False))

In [ ]:
faces = face_detector.detect_faces(cv2.cvtColor(to_face_img, cv2.COLOR_RGB2BGR))
face = faces[0]
face.landmarks = face_detector.get_landmarks(face)

In [ ]:
cfg['swap']['face_size'] = (64, 64)
swap_res = swap_faces(face, face_detector, cfg['swap'], face_generator)

In [ ]:
plt.imshow(cv2.cvtColor(swap_res, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
plt.imshow(cv2.cvtColor(swap_res, cv2.COLOR_BGR2RGB))
plt.show()

## Mask

In [ ]:
mask = utils.get_face_mask(faces[0], 'hull')

In [ ]:
# apply mask modifier (if specified in config)
erosion_size = None
if erosion_size:
    erosion_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,
                                               erosion_size)
    mask = cv2.erode(mask, erosion_kernel, iterations=1)
blur_size = 10
if blur_size > 0:
    mask = cv2.blur(mask, (blur_size, blur_size))


In [ ]:
plt.imshow(mask)
plt.show()

In [ ]:
plt.imshow(utils.get_face_mask(faces[0], 'hull'))
plt.show()

# Face Generation

In [ ]:
# generate random image
rnd_image = FaceGenerator.generate_random_img(None, (28, 28))
print(rnd_image.shape)

In [ ]:
plt.imshow(rnd_image)
plt.show()

In [ ]:
# random face transform
rnd_face = FaceGenerator.random_transform(from_face)
print(rnd_face.shape)

In [ ]:
plt.imshow(rnd_face)
plt.show()

In [ ]:
# random face warp
warped_face, target_face = FaceGenerator.random_warp(from_face_img)
print(warped_face.shape)

In [ ]:
plt.imshow(warped_face)
plt.show()

In [ ]:
plt.imshow(target_face)
plt.show()

## Autoencoder Generation

In [ ]:
aut_models_folder = models_folder / 'face_recognition' / 'deep_faceswap' / 'base_autoencoder' / 'trump_cage'

In [ ]:
aut_A, aut_B = autoencoder.get_autoencoders(str(aut_models_folder / 'v12k'))

In [ ]:
tmp_face = Face(to_face_img, None)
tmp_face.face_img = tmp_face.img

In [ ]:
gen_face = FaceGenerator.aue_generate_face(aut_A, tmp_face, (64, 64))
print(gen_face.shape)

In [ ]:
cv2.imshow("", gen_face)
#plt.show()

In [ ]:
face_generator = FaceGenerator.FaceGenerator(lambda x, y: FaceGenerator.aue_generate_face(aut_A, x, y))